# Exercise 5

## 1 Bias and variance of ridge regression

* True model: y = Xβ* + ε

* Zero-mean Gaussian noise: ε ~ N(0, σ^2)

* Centered features assumption: (1/N) * Σ Xi = 0

* Regularization parameter: τ ≥ 0

Prove that the expectation E[βτ] and covariance Cov[βτ] of the regularized solution βτ are given by:
        
        E[βτ] = (Sτ)⁻¹S β∗
        Cov[βτ] = (Sτ)⁻¹S (Sτ)⁻¹σ^2
 

**Step 1: Use the SVD of X:**

X = UΣVᵀ (where U and V are orthogonal matrices, Uᵀ = U⁻¹)

**Step 2: Compute S and Sτ:**

S = XᵀX = (UΣVᵀ)ᵀ(UΣVᵀ) = VΣᵀΣVᵀ

Sτ = XᵀX + τI = VΣᵀΣVᵀ + τI

**Step 3: Compute E[βτ]:**

βτ = argminβ (y - Xβ)ᵀ(y - Xβ) + τβᵀβ

= (yᵀ - βᵀXᵀ)(y - Xβ) + τβᵀβ

= yᵀy - yᵀXβ - βᵀXᵀy + βᵀXᵀXβ + τβᵀβ

To compute E[βτ], we can take the derivative of the function with respect to β and set it to zero:

0 = -Xᵀy + XᵀXβ + τβ

Xᵀy = XᵀXβ + τβ = β(XᵀX + τ)

β = (XᵀX + τI)⁻¹Xᵀy

E[βτ] = E[(XᵀX + τI)⁻¹Xᵀy] = (XᵀX + τI)⁻¹XᵀE[y]

Since we assumed that the noise ε is zero-mean, E[y] = Xβ*

E[βτ] = (XᵀX + τI)⁻¹XᵀXβ*

E[βτ] = (S + τI)⁻¹XᵀXβ*

Finally, since S = XᵀX and Sτ = XᵀX + τI, we have:

E[βτ] = (Sτ)⁻¹Sβ*

**Step 4: Compute Cov[βτ]:**

Cov[βτ] = Cov[(XᵀX + τI)⁻¹Xᵀy]

= (XᵀX + τI)⁻¹XᵀCov[y](XᵀX + τI)⁻¹

Since we assumed that the noise ε is zero-mean, Cov[y] = σ^2I:

Cov[βτ] = (XᵀX + τI)⁻¹Xᵀ(σ^2I)(XᵀX + τI)⁻¹

= (XᵀX + τI)⁻¹Xᵀσ^2I(XᵀX + τI)⁻¹

= σ^2(XᵀX + τI)⁻¹Xᵀ(XᵀX + τI)⁻¹

Finally, since S = XᵀX and Sτ = XᵀX + τI, we have:

Cov[βτ] = σ^2(Sτ)⁻¹Xᵀ(Sτ)⁻¹

## 2 LDA-Derivation from Least Squares Error

## 3 Automatic feature selection for LDA as regression

## 3.1 Implement Orthagonal Matching Pursuit

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from sklearn.datasets import load_digits
from sklearn import model_selection
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [5]:
# load digits data set
digits = load_digits()
data = digits["data"]
images = digits["images"]
target = digits["target"]
target_names = digits["target_names"]
print(f"data.shape = {data.shape}")
print(f"data.dtype = {data.dtype}")
print(f"images.shape = {images.shape}")
print(f"images.dtype= {images.dtype}")
print(f"target.shape = {target.shape}")
print(f"target.dtype = {target.dtype}")
print(f"target_names.shape = {target_names.shape}")
print(f"target_names.dtype= {target_names.dtype}")
print(f"target[:20] = {target[:20]}")

data.shape = (1797, 64)
data.dtype = float64
images.shape = (1797, 8, 8)
images.dtype= float64
target.shape = (1797,)
target.dtype = int64
target_names.shape = (10,)
target_names.dtype= int64
target[:20] = [0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9]


## 3.2 Classification with sparse LDA